# RT PLAN READER

In [7]:
import dicom

In [8]:
#plan = dicom.read_file("/home/gate/Documents/Phd/JupyterNotebook/data/RTplanReader/ROUAN/RP.201509329.STEC_PoumonD.dcm")
plan = dicom.read_file("/media/sf_jeremy/Documents/Phd/work/CRCT/work/tb/fullCreator/patient/IMART/dicom/RP.201509643.STEC_LSD2_6ph1.dcm")

#plan = dicom.read_file("/home/gate/Documents/Phd/JupyterNotebook/data/RTplanReader/imart/RP.201509643.STEC_LSD2_6ph1.dcm")

In [9]:
#print plan.BeamSequence[0].BlockSequence[0].BlockName
print(plan)

(0008, 0005) Specific Character Set              CS: 'ISO_IR 192'
(0008, 0012) Instance Creation Date              DA: '20190211'
(0008, 0013) Instance Creation Time              TM: '105527.404000'
(0008, 0016) SOP Class UID                       UI: RT Plan Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.246.352.71.5.950586737223.518882.20180413162347
(0008, 0020) Study Date                          DA: '20180322'
(0008, 0030) Study Time                          TM: '101048'
(0008, 0050) Accession Number                    SH: '103906086'
(0008, 0060) Modality                            CS: 'RTPLAN'
(0008, 0070) Manufacturer                        LO: 'Varian Medical Systems'
(0008, 0090) Referring Physician's Name          PN: 'MASSABEAU^CAROLE'
(0008, 1010) Station Name                        SH: 'SRVVARIAN-DB'
(0008, 1030) Study Description                   LO: 'TDM therapie SPC'
(0008, 103e) Series Description                  LO: 'Plans ARIA RadOnc'
(0008, 1048

In [78]:
#print(plan.FractionGroupSequence[0].ReferencedBeamSequence[3]) #[3])#BeamMeterset)
#print(plan.DoseReferenceSequence[0].DeliveryMaximumDose)
#print(plan.FractionGroupSequence[0].NumberofFractionsPlanned
print(plan.BeamSequence[0].PrimaryFluenceModeSequence[0].FluenceMode)

NON_STANDARD


## CREATE A PYTHON DICTIONARY OBJECT WITH USEFUL DATA OF RT PLAN

In [83]:
#Dictonary which will contains all useful data for RTplan
rt_plan_file = {} 

rt_plan_file['Name'] = plan.PatientName
rt_plan_file['ID'] = plan.PatientID
rt_plan_file['Birth'] = plan.PatientBirthDate

#Create a dictionnary for each beam in RT plan dictionnary
rt_plan_file['Beam'] = {}

for beam_ in plan.BeamSequence:
    
    if beam_.TreatmentDeliveryType == 'TREATMENT':
        
            beam_name=beam_.BeamName
            rt_plan_file['Beam'][beam_name] = {}  #Create a dictionnary for each (treatment Beam)
            rt_plan_file['Beam'][beam_name]['RadiationType'] = beam_.RadiationType
            Evalue=str(beam_.ControlPointSequence[0].NominalBeamEnergy)
            if plan.BeamSequence[3].PrimaryFluenceModeSequence[0].FluenceMode == 'NON_STANDARD':
                fluence_mode='FFF'
            else:
                fluence_mode='X'
            energy=Evalue+fluence_mode
            rt_plan_file['Beam'][beam_name]['Energy']=energy
            rt_plan_file['Beam'][beam_name]['BeamType']=beam_.BeamType


            xj = beam_.ControlPointSequence[0].BeamLimitingDevicePositionSequence[0].LeafJawPositions
            yj = beam_.ControlPointSequence[0].BeamLimitingDevicePositionSequence[1].LeafJawPositions

            rt_plan_file['Beam'][beam_name]['x_jaw'] = [float(val) for val in xj]
            rt_plan_file['Beam'][beam_name]['y_jaw'] = [float(val) for val in yj]
            
            rt_plan_file['Beam'][beam_name]['iso']=[float(val) for val in beam_.ControlPointSequence[0].IsocenterPosition]
            rt_plan_file['Beam'][beam_name]['iso'][1]=rt_plan_file['Beam'][beam_name]['iso'][1]*-1  #because rotation (x axis) in GATE
            rt_plan_file['Beam'][beam_name]['iso'][2]=rt_plan_file['Beam'][beam_name]['iso'][2]*-1  #because rotation (x axis) in GATE
            
            rt_plan_file['Beam'][beam_name]['rot_colli'] = float(plan.BeamSequence[0].ControlPointSequence[0].BeamLimitingDeviceAngle)
            rt_plan_file['Beam'][beam_name]['dir_rot_colli'] = plan.BeamSequence[0].ControlPointSequence[0].BeamLimitingDeviceRotationDirection
            
            #rt_plan_file['Beam'][beam_name]['DOSE'] = float(plan.DoseReferenceSequence[0].DeliveryMaximumDose)
            rt_plan_file['Beam'][beam_name]['UM'] = float(plan.FractionGroupSequence[0].ReferencedBeamSequence[0].BeamMeterset)
            rt_plan_file['Beam'][beam_name]['Fraction'] = float(plan.FractionGroupSequence[0].NumberofFractionsPlanned)
            
            if rt_plan_file['Beam'][beam_name]['RadiationType'] == 'ELECTRON':
                rt_plan_file['Beam'][beam_name]['applicatorID'] = beam_.ApplicatorSequence[0].ApplicatorID
                rt_plan_file['Beam'][beam_name]['applicator_type'] = beam_.ApplicatorSequence[0].ApplicatorType
                rt_plan_file['Beam'][beam_name]['SSD'] = beam_.SourceAxisDistance
                rt_plan_file['Beam'][beam_name]['insert'] = beam_.BlockSequence[0].BlockName
            
            
            rt_plan_file['Beam'][beam_name]['ControlPointSequence']={}
            
            if rt_plan_file['Beam'][beam_name]['BeamType'] == 'DYNAMIC':             

                for control_point_sequence_ in beam_.ControlPointSequence:
                                 
                    control_point_index=control_point_sequence_.ControlPointIndex
                    rt_plan_file['Beam'][beam_name]['ControlPointSequence'][control_point_index]={}
                    
                    if control_point_index == 0:
                        mlcx = control_point_sequence_.BeamLimitingDevicePositionSequence[2].LeafJawPositions
                        rt_plan_file['Beam'][beam_name]['ControlPointSequence'][control_point_index]['GantryRotationDirection'] = beam_.ControlPointSequence[0].GantryRotationDirection
                    else:
                        mlcx = control_point_sequence_.BeamLimitingDevicePositionSequence[0].LeafJawPositions
                        
                    rt_plan_file['Beam'][beam_name]['ControlPointSequence'][control_point_index]['MLCx'] = [float(val) for val in mlcx]
                    rt_plan_file['Beam'][beam_name]['ControlPointSequence'][control_point_index]['GantryAngle'] = round(float(control_point_sequence_.GantryAngle), 3)
                    rt_plan_file['Beam'][beam_name]['ControlPointSequence'][control_point_index]['CumulativeDoseCoef'] = round(float(control_point_sequence_.ReferencedDoseReferenceSequence[0].CumulativeDoseReferenceCoefficient), 4)
                    
            if rt_plan_file['Beam'][beam_name]['BeamType'] == 'STATIC':
                
                control_point_index=beam_.ControlPointSequence[0].ControlPointIndex
                rt_plan_file['Beam'][beam_name]['ControlPointSequence'][control_point_index]={}
                rt_plan_file['Beam'][beam_name]['ControlPointSequence'][control_point_index]['GantryAngle'] = round(float(beam_.ControlPointSequence[0].GantryAngle), 3)
                
                if len(beam_.ControlPointSequence[0].BeamLimitingDevicePositionSequence) > 2:
                
                    mlcx = beam_.ControlPointSequence[0].BeamLimitingDevicePositionSequence[2].LeafJawPositions
                    rt_plan_file['Beam'][beam_name]['ControlPointSequence'][control_point_index]['MLCx'] = [float(val) for val in mlcx]

study=str(str(rt_plan_file['Name'])).split('^')[0]
print("STUDY : " + study)
print("rt_plan_file dictionary  -->  Created")
print(rt_plan_file)

STUDY : IMART
rt_plan_file dictionary  -->  Created
{'Name': 'IMART^JEAN CLAUDE', 'ID': '201509643', 'Birth': '19431120', 'Beam': {'ARCSTE': {'RadiationType': 'PHOTON', 'Energy': '6FFF', 'BeamType': 'DYNAMIC', 'x_jaw': [-21.0, 20.0], 'y_jaw': [-25.0, 22.0], 'iso': [-64.903177054112, 49.623905929566, 15.0], 'rot_colli': 0.0, 'dir_rot_colli': 'NONE', 'UM': 1698.15195191855, 'Fraction': 5.0, 'ControlPointSequence': {'0': {'GantryRotationDirection': 'CC', 'MLCx': [-1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -8.95, -12.5, -13.56, -14.42, -15.52, -17.05, -18.95, -19.13, -19.17, -18.95, -16.3, -12.15, -9.17, -9.04, -8.97, -8.9, -8.52, -6.35, 2.1, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.29, -1.2

## Write RT plan Summary ".txt"

In [84]:
rt_plan_sumup_name="/home/gate/Documents/Phd/JupyterNotebook/data/RTplanReader/resume/rt_plan_"+ study +'.txt'
rt_plan_sumup=open(rt_plan_sumup_name, 'w') 

rt_plan_sumup.write("*********************** RT PLAN SUMARY ******************************"+'\n') 
rt_plan_sumup.write('\n')
rt_plan_sumup.write('\n')

rt_plan_sumup.write("GENERAL INFORMATION :"  + '\n' + '\n')
rt_plan_sumup.write("     PATIENT Name : " + str(rt_plan_file['Name']) + '\n')                       #Write  patient name
rt_plan_sumup.write("     PATIENT ID : " + str(rt_plan_file['ID']) + '\n')                    #write patient ID
rt_plan_sumup.write("     PATIENT Birth Date : " + str(rt_plan_file['Birth']) + '\n' + '\n' + '\n')  

rt_plan_sumup.write("TECHNICAL INFORMATION :"  + '\n' + '\n')
rt_plan_sumup.write("     Number of Treatment Beams : " + str(len(rt_plan_file['Beam'])) + '\n' + '\n')

beamsname=rt_plan_file['Beam'].keys()
for beams in beamsname:
    rt_plan_sumup.write("     Beams Name : " + beams + '\n')
    rt_plan_sumup.write("          Beams Type : " + rt_plan_file['Beam'][beams]['BeamType'] + '\n')
    rt_plan_sumup.write("          Radiation Type : " + rt_plan_file['Beam'][beams]['RadiationType'] + '\n')
    rt_plan_sumup.write("          Nominal Energy : " + str(rt_plan_file['Beam'][beams]['Energy']) + '\n')
    rt_plan_sumup.write("          X jaws opening : " + str(rt_plan_file['Beam'][beams]['x_jaw']) + '\n')
    rt_plan_sumup.write("          Y jaws opening : " + str(rt_plan_file['Beam'][beams]['y_jaw']) + '\n')
    rt_plan_sumup.write("          Isocenter position : " + str(rt_plan_file['Beam'][beams]['iso']) + '\n')
    #rt_plan_sumup.write("          Dose prescription : " + str(rt_plan_file['Beam'][beams]['DOSE']) + '\n')
    rt_plan_sumup.write("          UM (per fraction) : " + str(rt_plan_file['Beam'][beams]['UM']) + '\n')
    rt_plan_sumup.write("          Fraction planned : " + str(rt_plan_file['Beam'][beams]['Fraction']) + '\n')

    
    
    if rt_plan_file['Beam'][beams]['RadiationType'] =='ELECTRON':
        rt_plan_sumup.write("          Applicator ID : " + rt_plan_file['Beam'][beam_name]['applicatorID'] + '\n')
        rt_plan_sumup.write("          Applicator type : " + rt_plan_file['Beam'][beam_name]['applicator_type'] + '\n')
        rt_plan_sumup.write("          INSERT : " + rt_plan_file['Beam'][beam_name]['insert'] + '\n')
        rt_plan_sumup.write("          DSP : " + str(rt_plan_file['Beam'][beam_name]['SSD']) + '\n')
            
    if rt_plan_file['Beam'][beams]['BeamType'] =='STATIC':
        
        rt_plan_sumup.write("          Gantry Angle : " + str(rt_plan_file['Beam'][beams]['ControlPointSequence'][0]['GantryAngle']) + '\n')
        
    if rt_plan_file['Beam'][beams]['BeamType'] =='DYNAMIC':
        
        rt_plan_sumup.write("          Control Point index Number : " + str(len(rt_plan_file['Beam'][beams]['ControlPointSequence'])) + '\n')
        rt_plan_sumup.write("          GantryAngleRotation : " + rt_plan_file['Beam'][beams]['ControlPointSequence'][0]['GantryRotationDirection'] + '\n')
    
    
       # rt_plan_sumup.write('\n')

rt_plan_sumup.write(  '\n'  + '\n' + '\n' + '\n')
rt_plan_sumup.write("TECHNICAL INFORMATION DETAILED :"  + '\n' + '\n')


if rt_plan_file['Beam'][beams]['BeamType'] =='DYNAMIC' or len(beam_.ControlPointSequence[0].BeamLimitingDevicePositionSequence) > 2 :
    for beams in beamsname:
        rt_plan_sumup.write(" ===== Beams Name : " + beams + ' ======\n' + '\n')

        n_cpi=rt_plan_file['Beam'][beams]['ControlPointSequence'].keys()
        for cpi in n_cpi:
            rt_plan_sumup.write("   *** Control Point Index : " + str(cpi) + ' \n')
            rt_plan_sumup.write("      -> MLCx : " + str(rt_plan_file['Beam'][beams]['ControlPointSequence'][cpi]['MLCx']) + ' \n')
            rt_plan_sumup.write("      -> Gantry Angle : " + str(rt_plan_file['Beam'][beams]['ControlPointSequence'][cpi]['GantryAngle']) + ' \n')
            if 'CumulativeDoseCoef' in rt_plan_file['Beam'][beams]['ControlPointSequence'][0]:
                rt_plan_sumup.write("      -> Cumul Dose Coeff : " + str(rt_plan_file['Beam'][beams]['ControlPointSequence'][cpi]['CumulativeDoseCoef']) + ' \n' + ' \n')
        rt_plan_sumup.write('\n')
    
rt_plan_sumup.write('\n')

rt_plan_sumup.close()
print("rt_plan_"+ study +'.txt' '  -->'  '  Created')

rt_plan_IMART.txt  -->  Created


## Create mlc file for GATE

In [85]:
for beams_ in rt_plan_file['Beam'].keys():
    #print beams_
    if 'MLCx' in rt_plan_file['Beam'][beams_]['ControlPointSequence'][0]:
        i=1
        for beams_ in rt_plan_file['Beam'].keys():
            if 'MLCx' in rt_plan_file['Beam'][beams_]['ControlPointSequence'][0]:
                if rt_plan_file['Beam'][beams_]['BeamType'] == 'DYNAMIC':
                    mlc_file_name='dynamicMLC_'+ beams_ + '_' + study + '.mlc'
                    mlc_file = open('/home/gate/Documents/Phd/JupyterNotebook/data/RTplanReader/mlc/' + mlc_file_name, 'w')

                if rt_plan_file['Beam'][beams_]['BeamType'] == 'STATIC':
                    mlc_file_name='staticMLC_' + beams_ + '_' + study + '.mlc'
                    mlc_file = open(mlc_file_name, 'w')

                mlc_file.write('File Rev = H ' + '\n')
                mlc_file.write('Treatment = ' + rt_plan_file['Beam'][beams_]['BeamType'] + '\n')
                mlc_file.write('Last Name = ' + str(rt_plan_file['Name']) + '\n')
                mlc_file.write('First Name = ' + '\n')
                mlc_file.write('Patient ID = ' + str(rt_plan_file['ID']) + '\n')
                mlc_file.write('Number of Fields = ' + str(len(rt_plan_file['Beam'][beams_]['ControlPointSequence'])) + '\n')
                mlc_file.write('Model = Varian HD120' + '\n')
                mlc_file.write('Tolerance = 0.00' + '\n')
                mlc_file.write('\n')

                j=0
                for cpi in rt_plan_file['Beam'][beams_]['ControlPointSequence']:


                    mlc_file.write('Field = ' + 'Beam ' + str(i)+'.'+str(j) + '\n')
                    if rt_plan_file['Beam'][beams_]['BeamType'] == 'STATIC':
                        mlc_file.write('Index = ' + str(rt_plan_file['Beam'][beams_]['ControlPointSequence'][cpi]['GantryAngle']) + '\n')
                    if rt_plan_file['Beam'][beams_]['BeamType'] == 'DYNAMIC':
                        mlc_file.write('Index = ' + str(rt_plan_file['Beam'][beams_]['ControlPointSequence'][cpi]['GantryAngle']) + '\n')            

                    mlc_file.write('Carriage Group = 1 ' + '\n')
                    mlc_file.write('Operator = ' + '\n')
                    mlc_file.write('Collimator = 0.00' + '\n')  

                    ii=0
                    for leafs_ in rt_plan_file['Beam'][beams_]['ControlPointSequence'][cpi]['MLCx']:

                        if ii < 60:
                            mlc_file.write('Leaf ' + str(ii + 1) + 'A = ' + str(leafs_) + '\n')
                        else:
                            mlc_file.write('Leaf ' + str(ii - 59) + 'B = ' + str(leafs_) + '\n')

                        ii=ii+1

                    mlc_file.write('Note = 0' + '\n')
                    mlc_file.write('Shape = 0' + '\n')
                    mlc_file.write('Magnification  = 1.00' + '\n' + '\n')            
                    j=j+1            
            i=i+1
        print(mlc_file_name + ' --> created')
        mlc_file.close()
    else:
        print('No mlc in this beam plan')

dynamicMLC_ARCSTE_IMART.mlc --> created


## Create Clinic_Config.tb rt plan based
#### one file per beam

In [91]:
for beams_ in rt_plan_file['Beam'].keys():

    with open('//home/gate/Documents/Phd/JupyterNotebook/data/RTplanReader/config/clinic_config_temp.tb', 'r') as file:                                  #laod Config.tb   # Write JAW position in Config.tb file # At the moment only lfor the first Beam
        lines = file.readlines()
         
        x1_val = int(rt_plan_file['Beam'][beams_]['x_jaw'][0])
        x2_val = int(rt_plan_file['Beam'][beams_]['x_jaw'][1])
        y1_val = int(rt_plan_file['Beam'][beams_]['y_jaw'][0])
        y2_val = int(rt_plan_file['Beam'][beams_]['y_jaw'][1])
        
        if x1_val < 0:
            x1_val=int(x1_val*-1)
        if x2_val < 0:
            x2_val=int(x2_val*-1)
        if y1_val < 0:
            y1_val=int(y1_val*-1)
        if y2_val < 0:
            y2_val=int(y2_val*-1)
                
        lines[lines.index("=X1=\n")+1] = str(x1_val) +'\n'
        lines[lines.index("=X2=\n")+1] = str(x2_val) +'\n'
        lines[lines.index("=Y1=\n")+1] = str(y1_val) +'\n'
        lines[lines.index("=Y2=\n")+1] = str(y2_val) +'\n'
        
        if rt_plan_file['Beam'][beam_name]['BeamType'] == 'DYNAMIC':
             lines[lines.index("=DURATION=\n")+1] = str(len(rt_plan_file['Beam'][beam_name]['ControlPointSequence'].keys()))+'\n'
             lines[lines.index("=N_STEPS=\n")+1] = str(int(len(rt_plan_file['Beam'][beam_name]['ControlPointSequence'].keys()))+1) +'\n'

                
        if 'MLCx' in rt_plan_file['Beam'][beams_]['ControlPointSequence'][0]:   
            if rt_plan_file['Beam'][beam_name]['BeamType'] == 'DYNAMIC':        
                lines[lines.index("=MLC=\n")+1] = '2' +'\n'
            if rt_plan_file['Beam'][beam_name]['BeamType'] == 'STATIC' :
                lines[lines.index("=MLC=\n")+1] = '1' +'\n'
        else:
            lines[lines.index("=MLC=\n")+1] = '0' +'\n'
            
        if rt_plan_file['Beam'][beam_name]['BeamType'] == 'STATIC' :
                lines[lines.index("=GANTRY_STOP=\n")+1] = 'X' +'\n'
                lines[lines.index("=GANTRY=\n")+1] = str(int(rt_plan_file['Beam'][beams_]['ControlPointSequence'][0]['GantryAngle'])) +'\n'
        
        if rt_plan_file['Beam'][beam_name]['BeamType'] == 'DYNAMIC' :
            lines[lines.index("=GANTRY_STOP=\n")+1] = str(int(rt_plan_file['Beam'][beams_]['ControlPointSequence'][len(rt_plan_file['Beam'][beams_]['ControlPointSequence'])-1]['GantryAngle'])) +'\n'
            lines[lines.index("=GANTRY=\n")+1] = str(int(rt_plan_file['Beam'][beams_]['ControlPointSequence'][0]['GantryAngle'])) +'\n'
        
            if rt_plan_file['Beam'][beams_]['ControlPointSequence'][0]['GantryRotationDirection'] == 'CW':
                lines[lines.index("=ROTATION_DIRECTION=\n")+1] = '0' +'\n'
            else:
                lines[lines.index("=ROTATION_DIRECTION=\n")+1] = '1' +'\n'
        
        if 'MLCx' in rt_plan_file['Beam'][beams_]['ControlPointSequence'][0]:
            if rt_plan_file['Beam'][beams_]['BeamType'] == 'STATIC':
                lines[lines.index("=MLC_FILE=\n")+1]='staticMLC_' + beams_ + '_' + study + '.mlc' +'\n'
            else:    
                lines[lines.index("=MLC_FILE=\n")+1]='dynamicMLC_' + beams_ + '_' + study + '.mlc' +'\n' 
        else:
            lines[lines.index("=MLC_FILE=\n")+1]='XXX' +'\n'
        
        lines[lines.index("=PHANTOM=\n")+1]= study + '.mhd' +'\n'
        
        lines[lines.index("=ISO=\n")+1]=  str(rt_plan_file['Beam'][beams_]['iso']).replace('[', '').replace(']', '')
        
        lines[lines.index("=ACTOR=\n")+1]='XXX' +'\n'
        
        if rt_plan_file['Beam'][beams_]['RadiationType'] == 'ELECTRON':
            a='E'
        if rt_plan_file['Beam'][beams_]['RadiationType'] == 'PHOTON':
            a='X'
        
        lines[lines.index("=ENERGY=\n")+1]= str(rt_plan_file['Beam'][beams_]['Energy']) + a +'\n'
        
        if rt_plan_file['Beam'][beams_]['RadiationType'] == 'ELECTRON':
            lines[lines.index("=APPLICATOR=\n")+1] = rt_plan_file['Beam'][beams_]['applicatorID'] +'\n'
            if rt_plan_file['Beam'][beams_]['insert'] == 'PERSO' or rt_plan_file['Beam'][beams_]['insert'] == 'perso':
                lines[lines.index("=INSERT=\n")+1] = study+'.stl'
            else:
                lines[lines.index("=INSERT=\n")+1] = rt_plan_file['Beam'][beams_]['applicator_type'].replace('ELECTRON_', '') +'\n'
            
        print(beams_)
        lines[lines.index("=JOB_NAME=\n")+1] = beams_
        
    with open('/home/gate/Documents/Phd/JupyterNotebook/data/RTplanReader/config/ClinicConfig_' + beams_ +'_' +study + '.tb', 'w') as file:                                                                                                      #Write Config.tb with changes
        file.writelines(lines)
    print('ClinicConfig_'+ beams_ + '_' + study +'.tb' ' -->'  '  Created') 

ARCSTE
ClinicConfig_ARCSTE_IMART.tb -->  Created


In [88]:
a=len(rt_plan_file['Beam']['ARCSTE']['ControlPointSequence'].keys())
b=a+1
print(b)

92
